In [9]:
import requests
import json
import datetime
import time
from selectolax.parser import HTMLParser
import pandas as pd
import re
import os

In [10]:
path = '../Addition/'
os.listdir(path)

['zhihu_urls_addition.csv',
 '.DS_Store',
 'artificial intelligence',
 'Additional Web Crawler.ipynb',
 'AI',
 '.ipynb_checkpoints',
 'zhihu_urls.csv']

In [11]:
df = pd.DataFrame()
for t in ['AI/','artificial intelligence/']:
    for file in os.listdir(path+t):
        df = pd.concat([df, pd.read_csv(path+t+file)],axis=0)
        df.reset_index(inplace=True, drop=True)

In [12]:
df

,web-scraper-order,web-scraper-start-url,link,link-href
0,1674875917-2348,https://www.zhihu.com/search?type=content&q=AI...,不要低估AI面临的困境,https://zhuanlan.zhihu.com/p/79788463
1,1674875917-2349,https://www.zhihu.com/search?type=content&q=AI...,AI目前发展的瓶颈是什么?,https://www.zhihu.com/question/319906633/answe...
2,1674875917-2350,https://www.zhihu.com/search?type=content&q=AI...,AI现在还面临哪些挑战和难题（一）,https://zhuanlan.zhihu.com/p/543411665
3,1674875917-2351,https://www.zhihu.com/search?type=content&q=AI...,AI卡住了哪位大佬有解决的方法嘛,https://www.zhihu.com/zvideo/1360418316134182912
4,1674875917-2352,https://www.zhihu.com/search?type=content&q=AI...,如何看待AI的哲学困境？,https://www.zhihu.com/question/264445453/answe...
...,...,...,...,...
5891,1674878471-3686,https://www.zhihu.com/search?type=content&q=ar...,四大主流安卓手机厂商BootLoader中存在多个漏洞,https://zhuanlan.zhihu.com/p/29050515
5892,1674878471-3687,https://www.zhihu.com/search?type=content&q=ar...,学信息安全技术导致拿不到学校的学位证怎么办？,https://www.zhihu.com/question/32022810/answer...
5893,1674878471-3688,https://www.zhihu.com/search?type=content&q=ar...,亲手教的AI总是放心些,https://zhuanlan.zhihu.com/p/27483965
5894,1674878471-3689,https://www.zhihu.com/search?type=content&q=ar...,神经符号 AI，或为下一代 AIoT 的新解法,https://zhuanlan.zhihu.com/p/589716730


In [13]:
df.to_csv("zhihu_urls_addition.csv", encoding="utf-8-sig", index=False)

In [23]:
questionId = []
def getQuestionId(href):
    res = re.findall(r'https://www.zhihu.com/question/([0-9]+)',href)
    if(len(res)!=0):
        questionId.append(res[0])
for url in df['link-href']:
    getQuestionId(url)

In [24]:
len(questionId)

2691

In [25]:
questionId = list(set(questionId))

In [26]:
len(questionId)

1666

In [27]:
origin_questionId = []
def getQuestionId(href):
    res = re.findall(r'https://www.zhihu.com/question/([0-9]+)',href)
    if(len(res)!=0):
        origin_questionId.append(res[0])
for url in pd.read_csv('../Addition/zhihu_urls.csv')['question-href']:
    getQuestionId(url)

In [28]:
questionId = [i for i in questionId if i not in origin_questionId]

In [30]:
len(set(questionId))

1429

In [32]:
len(set(origin_questionId))

876

In [33]:
len(origin_questionId)

1600

In [10]:
def download_answer(question_id, start):
#    基于问题id获取单个问题下所有回答数据信息
    print(f"问题{question_id},第{start}条。")
    url = "https://www.zhihu.com/api/v4//questions/{}/answers".format(question_id)
    print(url)
    headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36',
               'referer':'https://www.zhihu.com/question/{}'.format(question_id),
              } 
    
    para = {
    'include':'data[*].is_normal,admin_closed_comment, \
    reward_info,is_collapsed,annotation_action,annotation_detail,\
    collapse_reason,is_sticky,collapsed_by,suggest_edit,comment_count,\
    can_comment,content,editable_content,attachment,voteup_count,\
    reshipment_settings,comment_permission,created_time,\
    updated_time,review_info,relevant_info,question,excerpt,is_labeled,\
    paid_info,paid_info_content,relationship.is_authorized,is_author,voting,is_thanked,\
    is_nothelp,is_recognized;data[*].mark_infos[*].url;data[*].author.follower_count,\
    vip_info,badge[*].topics;data[*].settings.table_of_content.enabled',
    'offset':start,
    'limit':20,
    'sort_by':'default',
    'platform':'desktop'
    }
    
    global data   

    response = requests.get(url,params=para,headers = headers)
    response.encoding = response.apparent_encoding
    if response.status_code == 200:
        res = json.loads(response.text)
        if res['data']:
            for answer in res['data']:
                author = answer['author']['name']
                fans = answer['author']['follower_count'] 
                content = HTMLParser(answer['content']).text()
                #content = answer['content']
                created_time = datetime.datetime.fromtimestamp(answer['created_time'])
                updated_time = datetime.datetime.fromtimestamp(answer['updated_time'])
                comment = answer['comment_count']
                voteup = answer['voteup_count']
                link = answer['url']
                row = {
                    'author':[author],
                    'fans_count':[fans],
                    'content':[content],
                    'created_time':[created_time],
                    'updated_time':[updated_time],
                    'comment_count':[comment],
                    'voteup_count':[voteup],
                    'url':[link]
                }
                data = data.append(pd.DataFrame(row),ignore_index=True)
            if len(res['data'])==20: 
                download_answer(question_id,start+20)
                time.sleep(1)
        else:
            return None
    else:
        return None

In [11]:
data = pd.DataFrame(columns=('author','fans_count','content','created_time','updated_time','comment_count','voteup_count','url'))

for i,id in enumerate(questionId):
    try:
        download_answer(id,0)
        print(str(i)+f'/{len(questionId)}')
    except: 
        continue

问题343256824,第0条。
https://www.zhihu.com/api/v4//questions/343256824/answers


/var/folders/29/27cgqwrx7_18wcd66rpffzlc0000gn/T/ipykernel_8471/1512429315.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(row),ignore_index=True)


0/1429
问题26911785,第0条。
https://www.zhihu.com/api/v4//questions/26911785/answers
1/1429
问题439453212,第0条。
https://www.zhihu.com/api/v4//questions/439453212/answers
2/1429
问题328094778,第0条。
https://www.zhihu.com/api/v4//questions/328094778/answers
3/1429
问题310002144,第0条。
https://www.zhihu.com/api/v4//questions/310002144/answers
4/1429
问题269262542,第0条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第20条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第40条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第60条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第80条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第100条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第120条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第140条。
https://www.zhihu.com/api/v4//questions/269262542/answers
问题269262542,第160条。
https://www.zhihu.com/api/v4//question

问题379814619,第800条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第820条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第840条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第860条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第880条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第900条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第920条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第940条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第960条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第980条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第1000条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第1020条。
https://www.zhihu.com/api/v4//questions/379814619/answers
问题379814619,第1040条。
https://www.zhihu.com/api/v4//questions/379814619/answ

82/1429
问题68398527,第0条。
https://www.zhihu.com/api/v4//questions/68398527/answers
问题68398527,第20条。
https://www.zhihu.com/api/v4//questions/68398527/answers
83/1429
问题559285560,第0条。
https://www.zhihu.com/api/v4//questions/559285560/answers
问题559285560,第20条。
https://www.zhihu.com/api/v4//questions/559285560/answers
84/1429
问题578702934,第0条。
https://www.zhihu.com/api/v4//questions/578702934/answers
85/1429
问题353261787,第0条。
https://www.zhihu.com/api/v4//questions/353261787/answers
问题353261787,第20条。
https://www.zhihu.com/api/v4//questions/353261787/answers
86/1429
问题362363880,第0条。
https://www.zhihu.com/api/v4//questions/362363880/answers
87/1429
问题551914040,第0条。
https://www.zhihu.com/api/v4//questions/551914040/answers
问题551914040,第20条。
https://www.zhihu.com/api/v4//questions/551914040/answers
问题551914040,第40条。
https://www.zhihu.com/api/v4//questions/551914040/answers
问题551914040,第60条。
https://www.zhihu.com/api/v4//questions/551914040/answers
问题551914040,第80条。
https://www.zhihu.com/api/v4//qu

...

问题41253586,第260条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第280条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第300条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第320条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第340条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第360条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第380条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第400条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第420条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第440条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第460条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第480条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第500条。
https://www.zhihu.com/api/v4//questions/41253586/answers
问题41253586,第520条。
https:/

1315/1429
问题320750959,第0条。
https://www.zhihu.com/api/v4//questions/320750959/answers
1316/1429
问题439891521,第0条。
https://www.zhihu.com/api/v4//questions/439891521/answers
1317/1429
问题333023887,第0条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第20条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第40条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第60条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第80条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第100条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第120条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第140条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第160条。
https://www.zhihu.com/api/v4//questions/333023887/answers
问题333023887,第180条。
https://www.zhihu.com/api/v4//questions/333023887/answers
1318/1429
问题412596343,第0条。
https://www.zhihu.com/api/v4/

问题557600073,第920条。
https://www.zhihu.com/api/v4//questions/557600073/answers
问题557600073,第940条。
https://www.zhihu.com/api/v4//questions/557600073/answers
1343/1429
问题456795182,第0条。
https://www.zhihu.com/api/v4//questions/456795182/answers
问题456795182,第20条。
https://www.zhihu.com/api/v4//questions/456795182/answers
问题456795182,第40条。
https://www.zhihu.com/api/v4//questions/456795182/answers
问题456795182,第60条。
https://www.zhihu.com/api/v4//questions/456795182/answers
问题456795182,第80条。
https://www.zhihu.com/api/v4//questions/456795182/answers
问题456795182,第100条。
https://www.zhihu.com/api/v4//questions/456795182/answers
1344/1429
问题303541452,第0条。
https://www.zhihu.com/api/v4//questions/303541452/answers
1345/1429
问题54955906,第0条。
https://www.zhihu.com/api/v4//questions/54955906/answers
1346/1429
问题566644880,第0条。
https://www.zhihu.com/api/v4//questions/566644880/answers
1347/1429
问题320138245,第0条。
https://www.zhihu.com/api/v4//questions/320138245/answers
1348/1429
问题30276886,第0条。
https://www.zhih

问题566830716,第460条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第480条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第500条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第520条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第540条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第560条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第580条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第600条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第620条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第640条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第660条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第680条。
https://www.zhihu.com/api/v4//questions/566830716/answers
问题566830716,第700条。
https://www.zhihu.com/api/v4//questions/566830716/answers

1391/1429
问题537337528,第0条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第20条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第40条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第60条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第80条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第100条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第120条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第140条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第160条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第180条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第200条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第220条。
https://www.zhihu.com/api/v4//questions/537337528/answers
问题537337528,第240条。
https://www.zhihu.com/api/v4//questions/537337528/ans

In [14]:
data.to_csv('data_zhihu_addition.csv',index=False, encoding='utf-8-sig')

In [24]:
import random

In [65]:
help(random.randint)

Help on method randint in module random:

randint(a, b) method of random.Random instance
    Return random integer in range [a, b], including both end points.



In [71]:
data.shape

(101694, 8)